In [1]:
import os
import pandas as pd
from google.cloud import storage

In [2]:
# # Usar no Google Colab
# from google.colab import auth
# auth.authenticate_user()
# path_folder_bronze = '/content/'

In [3]:
# Usar no VSCode
path_folder_silver = os.path.abspath('../data/temp/silver')
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../gcp_key.json"

In [4]:
def extract_year_month(date): # Função para extrair o ano e o mês de uma data
    return date.strftime('%y%m')

year_month = extract_year_month(pd.to_datetime('2017-06-01'))  # Exemplo de uso da função, como se fosse o dia 01 de junho de 2017
print(f'Year-Month: {year_month}')

Year-Month: 1706


In [5]:
def read_csv_cleaned(path_folder_silver, file_name_silver): # Função para ler o arquivo CSV limpo
    df = pd.read_csv(os.path.join(path_folder_silver, file_name_silver))
    
    print(f'Arquivo "{file_name_silver}" lido.')
    return df

file_name_silver = f'mta_{year_month}_cleaned.csv'

df = read_csv_cleaned(path_folder_silver, file_name_silver)
display(df.head(15))
display(df.info())

Arquivo "mta_1706_cleaned.csv" lido.


,RecordedAtTime,DirectionRef,PublishedLineName,VehicleRef,ScheduledArrivalTime,RecordedAtDate,DiffArrivalMins,RecordedTimeRange,ScheduledTimeRange
0,0 days 00:03:49,0,Bx10,NYCT_4223,0 days 00:00:53,2017-06-01,2,0,0
1,1 days 00:03:22,1,Bx1,NYCT_4710,0 days 23:59:38,2017-06-01,3,0,23
2,0 days 00:03:24,0,B31,NYCT_4611,0 days 00:08:00,2017-06-01,-4,0,0
3,0 days 00:03:52,1,B1,NYCT_7141,0 days 00:05:55,2017-06-01,-2,0,0
4,0 days 00:03:41,0,B6,NYCT_7158,0 days 00:05:00,2017-06-01,-1,0,0
5,0 days 00:03:36,0,B4,NYCT_445,0 days 00:01:49,2017-06-01,1,0,0
6,0 days 00:03:53,0,Bx5,NYCT_1280,0 days 00:02:00,2017-06-01,1,0,0
7,1 days 00:03:35,1,M11,NYCT_6372,0 days 23:57:00,2017-06-01,6,0,23
8,1 days 00:03:38,0,Bx32,NYCT_714,0 days 23:59:00,2017-06-01,4,0,23
9,1 days 00:03:46,0,B14,NYCT_7094,0 days 23:49:00,2017-06-01,14,0,23


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824936 entries, 0 to 1824935
Data columns (total 9 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   RecordedAtTime        object
 1   DirectionRef          int64 
 2   PublishedLineName     object
 3   VehicleRef            object
 4   ScheduledArrivalTime  object
 5   RecordedAtDate        object
 6   DiffArrivalMins       int64 
 7   RecordedTimeRange     int64 
 8   ScheduledTimeRange    int64 
dtypes: int64(4), object(5)
memory usage: 125.3+ MB


None

In [6]:
def change_data_types(df): # Função para alterar os tipos de dados das colunas do DataFrame
    df = df.astype({'PublishedLineName': 'string', 'VehicleRef': 'string'}) # Converte as colunas PublishedLineName e VehicleRef para string
    df['RecordedAtDate'] = pd.to_datetime(df['RecordedAtDate'])  # Converte a coluna RecordedAtDate para datetime
    df['RecordedAtTime'] = pd.to_timedelta(df['RecordedAtTime'])  # Converte a coluna RecordedAtTime para timedelta
    df['ScheduledArrivalTime'] = pd.to_timedelta(df['ScheduledArrivalTime'])  # Converte a coluna ScheduledArrivalTime para timedelta
    
    print("Tipos de dados das colunas alterados.")
    return df

df = change_data_types(df)
display(df.info())

Tipos de dados das colunas alterados.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1824936 entries, 0 to 1824935
Data columns (total 9 columns):
 #   Column                Dtype          
---  ------                -----          
 0   RecordedAtTime        timedelta64[ns]
 1   DirectionRef          int64          
 2   PublishedLineName     string         
 3   VehicleRef            string         
 4   ScheduledArrivalTime  timedelta64[ns]
 5   RecordedAtDate        datetime64[ns] 
 6   DiffArrivalMins       int64          
 7   RecordedTimeRange     int64          
 8   ScheduledTimeRange    int64          
dtypes: datetime64[ns](1), int64(4), string(2), timedelta64[ns](2)
memory usage: 125.3 MB


None